In [0]:
!git clone https://github.com/wlgh312/BSM_stylegan.git

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!mkdir /content/BSM_stylegan/raw_images

In [0]:
!pip install face_recognition

In [0]:
!pip install tqdm
!pip install typeguard
!pip install psutil
!pip install humanize
!pip install config
import cv2
import face_recognition
from IPython.display import Image
from google.colab import files
import pickle
import numpy as np
import PIL.Image
from PIL import Image
from shutil import copyfile
import config
from tqdm import tqdm
import matplotlib.pyplot as plt

In [0]:
import os
import sys
sys.path.append('/content/gdrive/My Drive/stylegan')
import dnnlib
import dnnlib.tflib as tflib#pre-trained networks
dnnlib.tflib.init_tf()

In [0]:
sys.path.insert(0, '/content/BSM_stylegan')
import ag_detection

In [0]:
entity_to_url={
        'faces':'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'
}
model_cache={}

In [0]:
def fetch_model(name):
  if model_cache.get(name):
    return model_cache[name]

  url = entity_to_url[name]
  with dnnlib.util.open_url(url, cache_dir='cache') as f:
    _G, _D, Gs = pickle.load(f)
    model_cache[name] = Gs
  return model_cache[name]

In [0]:
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=20)

In [0]:
seed=10000000
interpolate_dim=1#max:511
#mapping size
width=1#max:100
height=1
#image size
image_dim=100
interpolate=np.linspace(0, 30, width)
name='faces'
Gs=fetch_model(name)

In [0]:
def apply_latent_fudge(fudge, target_latent):
  copy=np.copy(target_latent)
  copy[interpolate_dim]+=fudge
  return copy

In [0]:
def v_linspace(i):
  v = np.array(list(map(lambda l: np.linspace(dst_dlatents[i][l], height), range(dst_dlatents.shape[1]))))
  v = v.transpose()
  return v

In [0]:
#dst:target image
def style_mixing(png, Gs, w, h, src_dlatents, dst_dlatents, style_ranges):
  print(png)
  src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
  dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)

  canvas = PIL.Image.new('RGB', (w * (len(src_dlatents) + 1), h*(len(dst_dlatents) + 1)), 'white')
  for col, src_image in enumerate(list(src_images)):
    canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), ((col + 1) * w, 0))
  for row, dst_image in enumerate(list(dst_images)):
    canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), (0, (row + 1) * h))
    row_dlatents = np.stack([dst_dlatents[row]] * len(src_dlatents))
    row_dlatents[:, style_ranges[row]] = src_dlatents[:, style_ranges[row]]
    row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=True, **synthesis_kwargs)
    for col, image in enumerate(list(row_images)):
      canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h))
  canvas.save(png)
  return canvas.resize((512, 512))

In [0]:
dst_image = '/content/BSM_stylegan/data/hugh_jackman.jpg'
mix_image = '/content/gdrive/My Drive/data/style_result/mix_result.png'

In [0]:
test_file=np.load('/content/BSM_stylegan/data/hugh_jackman_01.npy')

In [0]:
#Repeat until both age and gender are the same
for i in range(seed):
  #src_image=generate_image(seed)
  print(i)
  rnd=np.random.RandomState(i)
  init_latent1=rnd.randn(1, Gs.input_shape[1])[0]
  applier1 = lambda f: apply_latent_fudge(f, init_latent1)
  latent1=np.array(list(map(applier1, interpolate)))
  image = Gs.run(latent1, None, **synthesis_kwargs)
  img = PIL.Image.fromarray(image[0], 'RGB')
  img = img.resize((image_dim, image_dim), PIL.Image.ANTIALIAS)
  display(img)
  img.save('/content/BSM_stylegan/raw_images/seed.jpg')
  
  #src_image to npy
  %cd '/content/BSM_stylegan'
  !python npy_encoder.py seed.jpg --batch_size=2 --lr=0.01 --iterations=500 raw_images/ aligned_images/ generated_images/ latent_representation/
  src_file=np.load('/content/BSM_stylegan/latent_representation/seed_1.npy')
  #Style mix
  style_mixing(os.path.join('/content/gdrive/My Drive/data/style_result', 'mix_result.png'), Gs, w=1024, h=1024, src_dlatents=src_file.reshape((1, 18, 512)), dst_dlatents=test_file.reshape((1, 18, 512)), style_ranges=[range(6, 14)])
  #Detect age and gender of two photo
  ag_detection.test(dst_image)
  age1 = ag_detection.age
  gender1 = ag_detection.gender
  #Mixing photo crop
  img = Image.open('/content/gdrive/My Drive/data/style_result/mix_result.png')
  crop_img = img.crop((1024, 1024, 2048, 2048))
  crop_img.save('/content/gdrive/My Drive/data/style_result/mix.png')
  
  ag_detection.test('/content/gdrive/My Drive/data/style_result/mix.png')
  age2 = ag_detection.age
  gender2 = ag_detection.gender
  if age1 == age2 and gender1 == gender2:
    comp=0#same person
  else:
    comp=1#different person
  if comp == 0:
    break